In [ ]:
import sqlite3

with sqlite3.connect('netflix.sqlite') as conn:
    cursor = conn.cursor()

    cursor.execute('''CREATE TABLE IF NOT EXISTS actors (
                        id INTEGER PRIMARY KEY,
                        name TEXT
                    )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS movies (
                        movie_id INTEGER PRIMARY KEY,
                        title TEXT
                    )''')

    cursor.execute('''CREATE TABLE IF NOT EXISTS actor_movie (
                        actor_id INTEGER PRIMARY KEY,
                        actor_name TEXT,
                        movie_title TEXT,
                        FOREIGN KEY (actor_id) REFERENCES actors(id),
                        FOREIGN KEY (actor_name) REFERENCES actors(name),
                        FOREIGN KEY (movie_title) REFERENCES movies(title)
                    )''')

    # Создание таблицы с фильмами
    cursor.execute('INSERT INTO movies (movie_id, title) SELECT show_id, title FROM netflix_titles')

    # Создание таблицы с артёрами
    actors_list = []
    for all_actors in cursor.execute("SELECT `cast` FROM netflix_titles").fetchall():
        for actors in all_actors:
            for actor in actors.split(', '):
                actors_list.append(actor)
    actors_list = list(set(actors_list))
    act_id = 0
    for i in actors_list:
        if len(i) > 0:
            cursor.execute('INSERT INTO actors (id, name) VALUES (?, ?)', (act_id, i))
            act_id += 1

    # Создание таблицы актёр-фильмы
    def dict_factory(c, row):
        d = {}
        for idx, col in enumerate(c.description):
            d[col[0]] = row[idx]
        return d
    cursor.row_factory = dict_factory

    data = cursor.execute("SELECT * FROM netflix_titles").fetchall()
    actor_movie_data = {}
    for actor in actors_list:
        actor_movie_data[actor] = []
    for row in data:
        for actor in row['cast'].split(', '):
            actor_movie_data[actor].append(row['title'])

    for actor, movies in actor_movie_data.items():
        actor_id = cursor.execute('SELECT id FROM actors WHERE name = ?', (actor,)).fetchone()
        if actor_id is not None:
            cursor.execute('INSERT INTO actor_movie VALUES (?, ?, ?)', (actor_id['id'], actor, ', '.join(movies)))


    #print(cursor.execute("SELECT * FROM movies").fetchall())
    #print(cursor.execute("SELECT * FROM actors").fetchall())
    #print(cursor.execute("SELECT * FROM actor_movie").fetchall())
    conn.commit()
